In [2]:
import pandas as pd
import numpy as np
import pickle

# Collaborative-filtering 

In [ ]:
df = pd.read_csv("../data/raw/steam_reviews-ENGLISH.csv")

In [4]:
df.head()

,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
0,292030,The Witcher 3: Wild Hunt,85184605,english,"One of the best RPG's of all time, worthy of a...",1611379970,1611379970,True,0,0,...,True,False,False,76561199054755373,5,3,5587.0,3200.0,5524.0,1.611384e+09
1,292030,The Witcher 3: Wild Hunt,85184171,english,"good story, good graphics. lots to do.",1611379264,1611379264,True,0,0,...,True,False,False,76561198170193529,11,1,823.0,823.0,823.0,1.611379e+09
2,292030,The Witcher 3: Wild Hunt,85184064,english,"dis gud,",1611379091,1611379091,True,0,0,...,True,False,False,76561198119302812,27,2,4192.0,3398.0,4192.0,1.611352e+09
3,292030,The Witcher 3: Wild Hunt,85180436,english,favorite game of all time cant wait for the Ne...,1611373086,1611373086,True,0,0,...,True,False,False,76561198065591528,33,1,23329.0,177.0,23329.0,1.611219e+09
4,292030,The Witcher 3: Wild Hunt,85179753,english,Why wouldn't you get this,1611371978,1611371978,True,0,0,...,True,False,False,76561198996835044,131,2,8557.0,2004.0,8557.0,1.611371e+09


In [5]:
df.shape

(9635437, 22)

In [22]:
# sparse matrix

user_game_mat = pd.DataFrame(int(0),index=df['author.steamid'].unique() ,columns=df['app_name'].unique())

reviewers, games= df['author.steamid'], df['app_name']
for r, g in zip(reviewers,games):
    try:
        user_game_mat.loc[int(r),g] = int(1)
    except:
        continue
user_game_mat

,The Witcher 3: Wild Hunt,Half-Life,Counter-Strike: Source,Half-Life 2: Episode Two,Portal 2,X Rebirth,Garry's Mod,Sid Meier's Civilization V,Dead by Daylight,Sid Meier's Civilization VI,...,Thimbleweed Park,Toukiden 2,Dead Rising 4,X-Plane 11,Styx: Shards of Darkness,Train Sim World® 2020,DRAGON QUEST HEROES™ II,Steel Division: Normandy 44,DiRT 4,Half-Life: Alyx
76561198010955177,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
76561198332696736,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
76561198098346837,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
76561199013170832,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
76561198085431842,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76561198199668210,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
76561198061774352,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
76561197983355007,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
76561198099785821,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


## Assign ratings

In [4]:
median_df = df.groupby('app_name')["author.playtime_forever"].median()

In [5]:
median_df

app_name
20XX                                      1182.0
A Hat in Time                             1590.0
A Short Hike                               231.0
A Way Out                                  378.0
ARK: Survival Evolved                    14068.0
                                          ...   
XCOM 2                                    6768.0
Yakuza 0                                  3504.0
Youtubers Life                            1060.0
theHunter: Call of the Wild™              2888.0
三国群英传8 Heroes of the Three Kingdoms 8      655.0
Name: author.playtime_forever, Length: 315, dtype: float64

In [6]:
median_df = pd.DataFrame(median_df).reset_index()
median_df.columns = ['app_name', 'median_playtime']

In [7]:
median_df

,app_name,median_playtime
0,20XX,1182.0
1,A Hat in Time,1590.0
2,A Short Hike,231.0
3,A Way Out,378.0
4,ARK: Survival Evolved,14068.0
...,...,...
310,XCOM 2,6768.0
311,Yakuza 0,3504.0
312,Youtubers Life,1060.0
313,theHunter: Call of the Wild™,2888.0


In [8]:
df = pd.merge(df, median_df,right_on='app_name', left_on='app_name')

In [9]:
df.head()

,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played,median_playtime
0,292030,The Witcher 3: Wild Hunt,85184605,english,"One of the best RPG's of all time, worthy of a...",1611379970,1611379970,True,0,0,...,False,False,76561199054755373,5,3,5587.0,3200.0,5524.0,1.611384e+09,7483.0
1,292030,The Witcher 3: Wild Hunt,85184171,english,"good story, good graphics. lots to do.",1611379264,1611379264,True,0,0,...,False,False,76561198170193529,11,1,823.0,823.0,823.0,1.611379e+09,7483.0
2,292030,The Witcher 3: Wild Hunt,85184064,english,"dis gud,",1611379091,1611379091,True,0,0,...,False,False,76561198119302812,27,2,4192.0,3398.0,4192.0,1.611352e+09,7483.0
3,292030,The Witcher 3: Wild Hunt,85180436,english,favorite game of all time cant wait for the Ne...,1611373086,1611373086,True,0,0,...,False,False,76561198065591528,33,1,23329.0,177.0,23329.0,1.611219e+09,7483.0
4,292030,The Witcher 3: Wild Hunt,85179753,english,Why wouldn't you get this,1611371978,1611371978,True,0,0,...,False,False,76561198996835044,131,2,8557.0,2004.0,8557.0,1.611371e+09,7483.0


In [12]:
######################## Assume the ratings  ###############
def get_ratings(df_col):
    df_col['Rating'] = 0
    for i in range(len(df_col)):
        if df_col['author.playtime_forever'][i]>= df_col['median_playtime'][i]:
            df_col['Rating'][i] = 5
    #else:
     #   df_col['Rating'][i] = 2
        elif df_col['median_playtime'][i]>df_col['author.playtime_forever'][i]>= df_col['median_playtime'][i]*0.8:
            df_col['Rating'][i] = 4
        elif df_col['median_playtime'][i]*0.8>df_col['author.playtime_forever'][i]>= df_col['median_playtime'][i]*0.5:
            df_col['Rating'][i] = 3
        elif df_col['median_playtime'][i]*0.5>df_col['author.playtime_forever'][i]>= df_col['median_playtime'][i]*0.1:
            df_col['Rating'][i] = 2
        elif df_col['median_playtime'][i]*0.1>df_col['author.playtime_forever'][i]:
            df_col['Rating'][i] = 1
    return df_col

In [13]:
get_ratings(df)

/home/abdulium/miniconda3/envs/t5/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/home/abdulium/miniconda3/envs/t5/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/abdulium/miniconda3/envs/t5/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/abdulium

,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played,median_playtime,Rating
0,292030,The Witcher 3: Wild Hunt,85184605,english,"One of the best RPG's of all time, worthy of a...",1611379970,1611379970,True,0,0,...,False,76561199054755373,5,3,5587.0,3200.0,5524.0,1.611384e+09,7483.0,3
1,292030,The Witcher 3: Wild Hunt,85184171,english,"good story, good graphics. lots to do.",1611379264,1611379264,True,0,0,...,False,76561198170193529,11,1,823.0,823.0,823.0,1.611379e+09,7483.0,2
2,292030,The Witcher 3: Wild Hunt,85184064,english,"dis gud,",1611379091,1611379091,True,0,0,...,False,76561198119302812,27,2,4192.0,3398.0,4192.0,1.611352e+09,7483.0,3
3,292030,The Witcher 3: Wild Hunt,85180436,english,favorite game of all time cant wait for the Ne...,1611373086,1611373086,True,0,0,...,False,76561198065591528,33,1,23329.0,177.0,23329.0,1.611219e+09,7483.0,5
4,292030,The Witcher 3: Wild Hunt,85179753,english,Why wouldn't you get this,1611371978,1611371978,True,0,0,...,False,76561198996835044,131,2,8557.0,2004.0,8557.0,1.611371e+09,7483.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9635432,546560,Half-Life: Alyx,65645167,english,best game ever\n,1584986750,1584986750,True,0,0,...,False,76561198817267089,86,8,1055.0,14.0,45.0,1.611138e+09,995.0,5
9635433,546560,Half-Life: Alyx,65645125,english,Holy balls,1584986704,1584986704,True,0,0,...,False,76561197967033932,431,7,432.0,0.0,29.0,1.586628e+09,995.0,2
9635434,546560,Half-Life: Alyx,65645115,english,"A tripod thing bears down on you, it looks imm...",1584986693,1584986693,True,0,0,...,False,76561198046495482,70,10,3296.0,15.0,24.0,1.611080e+09,995.0,5
9635435,546560,Half-Life: Alyx,65645100,english,Honestly this is the best vr game ever,1584986673,1584986673,True,0,0,...,False,76561198176379749,79,17,2654.0,0.0,34.0,1.591634e+09,995.0,5


In [14]:
# pickle rating
pickle.dump(df, open("df_rating.pickle", 'wb'))

In [2]:
# open pickle
df = pickle.load(open("df_rating.pickle", 'rb'))

In [ ]:
# sparse matrix

user_game_mat_ratings = pd.DataFrame(int(0),index=df['author.steamid'].unique() ,columns=df['app_name'].unique())

reviewers, games, ratings = df['author.steamid'], df['app_name'], df['Rating']
for rev, game, rating in zip(reviewers,games, ratings):
    try:
        user_game_mat_ratings.loc[int(rev),game] = rating
    except:
        continue
user_game_mat_ratings

In [1]:
user_game_mat_ratings

NameError: name 'user_game_mat_ratings' is not defined

In [41]:
sum(user_game_mat_ratings["The Witcher 3: Wild Hunt"])

0

In [2]:
df.columns

NameError: name 'df' is not defined

In [3]:
cols = ['app_name',
       'author.steamid',
       'author.playtime_forever',
       'median_playtime',
       'Rating']

In [4]:
df.drop(columns=df.columns.difference(cols), inplace=True)

In [5]:
df.head()

,app_name,author.steamid,author.playtime_forever,median_playtime,Rating
0,The Witcher 3: Wild Hunt,76561199054755373,5587.0,7483.0,3
1,The Witcher 3: Wild Hunt,76561198170193529,823.0,7483.0,2
2,The Witcher 3: Wild Hunt,76561198119302812,4192.0,7483.0,3
3,The Witcher 3: Wild Hunt,76561198065591528,23329.0,7483.0,5
4,The Witcher 3: Wild Hunt,76561198996835044,8557.0,7483.0,5


In [6]:
# pickle rating_no_cols
pickle.dump(df, open("rating_no_cols.pickle", 'wb'))

In [2]:
#load df
df = pickle.load(open('rating_no_cols.pickle', 'rb'))

In [3]:
sparse = pd.pivot_table(df, index='author.steamid', columns='app_name', values='Rating').fillna(0).convert_dtypes()

In [4]:
sparse

,20XX,A Hat in Time,A Short Hike,A Way Out,ARK: Survival Evolved,ATLAS,Age of Empires II (2013),Age of Empires: Definitive Edition,American Truck Simulator,Among Us,...,We Were Here Too,Wolfenstein: Youngblood,X Rebirth,X-Blades,X-Plane 11,XCOM 2,Yakuza 0,Youtubers Life,theHunter: Call of the Wild™,三国群英传8 Heroes of the Three Kingdoms 8
author.steamid,,,,,,,,,,,,,,,,,,,,,
76561197960265730,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
76561197960265745,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
76561197960265747,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
76561197960265778,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
76561197960265781,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76561199132972128,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
76561199132989128,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
76561199133026644,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# pickle sparse
pickle.dump(sparse, open("ratings_sparse.pickle", 'wb'))

In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
# load sparse
df = pickle.load(open("ratings_sparse.pickle", 'rb'))

In [3]:
df = df.iloc[:100000,:]

In [4]:
from sklearn.decomposition import PCA

In [26]:
# lsa = TruncatedSVD(n_components=10)

In [27]:
# lsa.fit_transform(df)

array([[ 3.67084767e-02,  5.02015733e-02,  2.70172825e-02, ...,
        -8.78361550e-03,  3.02714705e-02, -3.04387594e-02],
       [ 1.21550134e+00,  2.41622620e+00, -3.75762769e+00, ...,
         9.32807447e-02, -3.21721940e-01, -6.19655790e-01],
       [ 1.46312890e-01,  2.85092152e-01, -1.82869784e-02, ...,
        -8.77763343e-01, -9.82027725e-01, -3.27404611e-01],
       ...,
       [ 1.73539197e-02,  1.41303414e-02,  3.73589120e-03, ...,
         2.84114033e-03, -8.24595609e-03, -6.36117214e-04],
       [ 5.20165933e-01,  9.86120485e-01, -6.32942170e-01, ...,
        -1.32673466e-01,  4.89950312e-01, -3.84434419e-01],
       [ 8.03606417e-02,  1.28257412e-01,  2.90628024e-02, ...,
        -2.01117388e-02,  2.55819392e-02,  6.63234371e-03]])

In [5]:
pca = PCA(n_components=5)

In [6]:
pca.fit_transform(df)

array([[-0.29317401, -0.48012321,  0.02259808, -0.2935848 , -0.13582071],
       [-0.50096283,  3.59853228, -2.16480116, -2.27044611, -1.16002035],
       [-0.34795   , -0.37367962, -0.00609273, -0.39844392, -0.12398233],
       ...,
       [-0.28374774, -0.48510991, -0.00543974, -0.31340835, -0.14190925],
       [-0.40397604,  0.61160722, -0.36313065, -0.55074775,  0.0941829 ],
       [-0.29620937, -0.39622334,  0.05600211, -0.24781608, -0.13493553]])

In [7]:
pca.explained_variance_

array([1.34706288, 0.85132487, 0.76016686, 0.75177792, 0.65565717])

In [12]:
game_genre_mat = pd.DataFrame(pca.components_, index=[i for i in range(0,5)], columns=pca.feature_names_in_).T

In [13]:
game_genre_mat.iloc[:10,:]

,0,1,2,3,4
20XX,-0.000828,0.001519,0.001436,0.002440,-0.000141
A Hat in Time,-0.001220,0.006707,0.003071,0.006367,-0.000837
A Short Hike,-0.001245,0.002447,0.000607,0.001846,-0.001110
A Way Out,0.000311,0.001474,0.000310,0.001867,0.001476
ARK: Survival Evolved,-0.002260,0.086996,-0.013836,-0.033833,0.178669
ATLAS,0.005092,0.011223,-0.007712,-0.010209,0.015680
Age of Empires II (2013),-0.000431,0.001588,0.006693,0.003802,0.004457
Age of Empires: Definitive Edition,0.000810,0.001108,0.001114,0.001643,0.000754
American Truck Simulator,-0.001052,0.009208,-0.001181,0.002922,0.009685
Among Us,0.000548,-0.003357,-0.002889,0.004032,0.009842
